In [1]:
from datetime import datetime
from influxdb_client import InfluxDBClient

### Getting Started with InfluxDB and Pandas
[Getting Started with InfluxDB and Pandas](https://www.influxdata.com/blog/getting-started-with-influxdb-and-pandas/)
### influxdb-client-python
[influxdb-client-python](https://github.com/influxdata/influxdb-client-python)
### Getting Started with Python and InfluxDB v2.0
[Getting Started with Python and InfluxDB v2.0](https://www.influxdata.com/blog/getting-started-with-python-and-influxdb-v2-0/)
### How to ingest DataFrame with default tags
[How to ingest DataFrame with default tags](https://github.com/influxdata/influxdb-client-python/blob/master/examples/ingest_dataframe_default_tags.py)
### How to ingest large DataFrame by splitting into chunks
[How to ingest large DataFrame by splitting into chunks](https://github.com/influxdata/influxdb-client-python/blob/master/examples/ingest_large_dataframe.py)

influx delete --bucket Tangara --start 1970-01-01T00:00:00Z --stop $(date +"%Y-%m-%dT%H:%M:%SZ") --predicate '_measurement="PM25"'
https://docs.influxdata.com/influxdb/cloud/reference/cli/influx/v1/dbrp/create/


In [2]:
# Function to Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
def ingesting_influxdb_1_8(data_sensors, measurement_name):
    # Secrets
    url = 'http://192.168.1.51:8086'
    username = 'tangara'
    password = 'sebaxtian'
    token = f'{username}:{password}'
    database = 'Tangara'
    retention_policy = 'autogen'
    bucket = f'{database}/{retention_policy}'
    org = "Tangara"

    # For each Data Sensor
    for column in data_sensors.columns[1:]:
        # For each Data Sensor
        tangara_X = data_sensors[['DATETIME', column]].copy()
        tangara_X['FIELD'] = column
        """
        Ingest DataFrame
        """
        #print()
        #print(f"=== Ingesting DataFrame {column} via batching API ===")
        #print()
        startTime = datetime.now()

        with InfluxDBClient(url=url, token=token, org=org) as client:
            """
            Use batching API
            """
            with client.write_api() as write_api:
                write_api.write(bucket=bucket, record=tangara_X,
                                data_frame_timestamp_column='DATETIME',
                                data_frame_tag_columns=['FIELD'],
                                data_frame_measurement_name=measurement_name,
                                data_frame_timestamp_timezone='America/Bogota')
                #print()
                #print(f"Wait to finishing ingesting DataFrame {column}...")
                #print()

        #print()
        #print(f'Import finished in: {datetime.now() - startTime}')
        #print()

In [3]:
# Function to Ingesting each Tangara Sensor DataFrame to InfluxDB
def ingesting_influxdb(data_sensors, measurement_name):
    # Secrets
    # You can generate an API token from the "API Tokens Tab" in the UI
    url = 'http://localhost:8086'
    token = "NfSGn5XfOZrVYLUtpijvo4c4FPuOLJ4yZ3-Ji7LLwS3M1y54XlfAI6qLUzhmmDvqd1NbIZG57jB0sMYybPGASg=="
    org = "Tangara"
    bucket = "Tangara"
    
    # For each Data Sensor
    for column in data_sensors.columns[1:]:
        # For each Data Sensor
        tangara_X = data_sensors[['DATETIME', column]].copy()
        tangara_X['FIELD'] = column
        """
        Ingest DataFrame
        """
        #print()
        #print(f"=== Ingesting DataFrame {column} via batching API ===")
        #print()
        startTime = datetime.now()

        with InfluxDBClient(url=url, token=token, org=org) as client:
            """
            Use batching API
            """
            with client.write_api() as write_api:
                write_api.write(bucket=bucket, record=tangara_X,
                                data_frame_timestamp_column='DATETIME',
                                data_frame_tag_columns=['FIELD'],
                                data_frame_measurement_name=measurement_name,
                                data_frame_timestamp_timezone='America/Bogota')
                #print()
                #print(f"Wait to finishing ingesting DataFrame {column}...")
                #print()

        #print()
        #print(f'Import finished in: {datetime.now() - startTime}')
        #print()

---

# Extraction - PM25 Raw

In [4]:
# Kedro Catalog
pm25_raw = catalog.load('pm25_raw')
# Data Frame Sensors
pm25_raw.head()

2022-09-17 17:07:16,215 - kedro.io.data_catalog - INFO - Loading data from `pm25_raw` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-16T17:06:30,19,18.0,23.0,21.0,22.0,20.0,4.0,NaN,31.0,39.0,19.0,23.0
1,2022-09-16T17:07:00,20,16.0,21.0,23.0,21.0,19.0,4.0,14.0,25.0,39.0,19.0,22.0
2,2022-09-16T17:07:30,18,17.0,22.0,22.0,22.0,18.0,4.0,15.0,32.0,40.0,19.0,22.0
3,2022-09-16T17:08:00,19,17.0,21.0,20.0,22.0,18.0,4.0,11.0,24.0,40.0,18.0,22.0
4,2022-09-16T17:08:30,18,18.0,22.0,21.0,23.0,18.0,4.0,15.0,28.0,39.0,18.0,22.0


In [5]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_raw, 'PM25_RAW')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_raw, 'PM25_RAW')

# Extraction - PM25 Clean

In [6]:
# Kedro Catalog
pm25_clean = catalog.load('pm25_clean')
# Data Frame Sensors
pm25_clean.head()

2022-09-17 17:07:26,886 - kedro.io.data_catalog - INFO - Loading data from `pm25_clean` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-16T17:06:30,19.0,18.0,23.0,21.0,22.0,20.0,4.0,NaN,31.0,NaN,19.0,23.0
1,2022-09-16T17:07:00,20.0,16.0,21.0,23.0,21.0,19.0,4.0,14.0,25.0,NaN,19.0,22.0
2,2022-09-16T17:07:30,18.0,17.0,22.0,22.0,22.0,18.0,4.0,15.0,32.0,NaN,19.0,22.0
3,2022-09-16T17:08:00,19.0,17.0,21.0,20.0,22.0,18.0,4.0,11.0,24.0,NaN,18.0,22.0
4,2022-09-16T17:08:30,18.0,18.0,22.0,21.0,23.0,18.0,4.0,15.0,28.0,NaN,18.0,22.0


In [7]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_clean, 'PM25_CLEAN')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_clean, 'PM25_CLEAN')

# Extraction - PM25 Last Hour

In [8]:
# Kedro Catalog
pm25_last_hour = catalog.load('pm25_last_hour')
# Data Frame Sensors
pm25_last_hour.head()

2022-09-17 17:07:35,783 - kedro.io.data_catalog - INFO - Loading data from `pm25_last_hour` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-16 18:06:00,21.920354,23.034783,20.327586,9.904348,19.401709,17.070796,2.252174,7.765766,12.153846,9.618182,8.732759,23.068966
1,2022-09-16 19:06:00,11.983193,13.589744,17.282051,2.427350,5.336134,2.754237,0.051724,1.170940,0.420168,6.965812,1.358974,9.228070
2,2022-09-16 20:06:00,5.537736,15.343137,19.632075,4.237624,3.039216,2.030000,0.010101,0.650485,0.273585,10.695238,1.009615,5.754902
3,2022-09-16 21:06:00,10.586207,20.815789,26.201835,11.567797,10.563025,4.478992,0.008403,1.362832,0.209091,9.224138,1.117647,9.104348
4,2022-09-16 22:06:00,17.401709,22.244681,15.617391,10.096491,15.500000,6.767241,0.465517,2.697248,1.521368,10.808696,2.367521,18.102564


In [9]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_last_hour, 'PM25_LAST_HOUR')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_last_hour, 'PM25_LAST_HOUR')

# Extraction - PM25 Last 8 Hours

In [10]:
# Kedro Catalog
pm25_last_8h = catalog.load('pm25_last_8h')
# Data Frame Sensors
pm25_last_8h.head()

2022-09-17 17:07:39,204 - kedro.io.data_catalog - INFO - Loading data from `pm25_last_8h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 01:06:00,15.828050,18.207241,18.952022,9.977007,12.463637,8.683103,0.921913,3.658638,4.852570,10.399530,4.237239,14.126724
1,2022-09-17 09:06:00,17.684108,21.383742,24.125651,15.323023,18.279659,10.940561,2.416175,9.104221,16.963288,16.657168,11.674237,19.105752
2,2022-09-17 17:06:00,9.026863,10.079109,12.520018,9.772327,9.522251,7.150683,2.458492,7.996912,14.932714,12.068878,10.080400,10.816270


In [11]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_last_8h, 'PM25_LAST_8H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_last_8h, 'PM25_LAST_8H')

# Extraction - PM25 Last 12 Hours

In [12]:
# Kedro Catalog
pm25_last_12h = catalog.load('pm25_last_12h')
# Data Frame Sensors
pm25_last_12h.head()

2022-09-17 17:07:43,110 - kedro.io.data_catalog - INFO - Loading data from `pm25_last_12h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 05:06:00,16.690228,18.861801,20.461033,11.327020,14.619715,8.377660,1.151470,4.259047,6.678126,11.527263,5.361229,15.830069
1,2022-09-17 17:06:00,11.669120,14.251593,16.604095,12.054552,12.223982,9.471904,2.712917,9.580800,17.820923,14.556455,11.966688,13.535762


In [13]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_last_12h, 'PM25_LAST_12H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_last_12h, 'PM25_LAST_12H')

# Extraction - PM25 Last 24 Hours

In [14]:
# Kedro Catalog
pm25_last_24h = catalog.load('pm25_last_24h')
# Data Frame Sensors
pm25_last_24h.head()

2022-09-17 17:07:46,930 - kedro.io.data_catalog - INFO - Loading data from `pm25_last_24h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 17:06:00,14.179674,16.556697,18.532564,11.690786,13.421849,8.924782,1.932193,6.919924,12.249524,13.041859,8.663959,14.682916


In [15]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(pm25_last_24h, 'PM25_LAST_24H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(pm25_last_24h, 'PM25_LAST_24H')

---

# Extraction - AQI Instant

In [16]:
# Kedro Catalog
aqi_instant = catalog.load('aqi_instant')
# Data Frame Sensors
aqi_instant.head()

2022-09-17 17:07:50,878 - kedro.io.data_catalog - INFO - Loading data from `aqi_instant` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-16T17:06:30,66.0,64.0,74.0,70.0,72.0,68.0,17.0,NaN,91.0,NaN,66.0,74.0
1,2022-09-16T17:07:00,68.0,60.0,70.0,74.0,70.0,66.0,17.0,55.0,79.0,NaN,66.0,72.0
2,2022-09-16T17:07:30,64.0,62.0,72.0,72.0,72.0,64.0,17.0,58.0,93.0,NaN,66.0,72.0
3,2022-09-16T17:08:00,66.0,62.0,70.0,68.0,72.0,64.0,17.0,46.0,77.0,NaN,64.0,72.0
4,2022-09-16T17:08:30,64.0,64.0,72.0,70.0,74.0,64.0,17.0,58.0,85.0,NaN,64.0,72.0


In [17]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(aqi_instant, 'AQI_INSTANT')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(aqi_instant, 'AQI_INSTANT')

# Extraction - AQI Last Hour

In [18]:
# Kedro Catalog
aqi_last_hour = catalog.load('aqi_last_hour')
# Data Frame Sensors
aqi_last_hour.head()

2022-09-17 17:08:00,202 - kedro.io.data_catalog - INFO - Loading data from `aqi_last_hour` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-16 18:06:00,72,74,69,42,67,62,10,33,52,40,37,75
1,2022-09-16 19:06:00,50,55,62,10,23,12,1,5,2,30,6,39
2,2022-09-16 20:06:00,23,58,67,18,13,9,0,3,2,45,5,25
3,2022-09-16 21:06:00,45,70,81,49,45,19,0,6,1,39,5,38
4,2022-09-16 22:06:00,63,73,59,43,59,29,3,12,7,46,10,64


In [19]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(aqi_last_hour, 'AQI_LAST_HOUR')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(aqi_last_hour, 'AQI_LAST_HOUR')

2022-09-17 17:08:00,890 - influxdb_client.client.write_api - ERROR - The batch item wasn't processed successfully because: (422)
Reason: Unprocessable Entity
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.4.0', 'X-Platform-Error-Code': 'unprocessable entity', 'Date': 'Sat, 17 Sep 2022 22:08:00 GMT', 'Content-Length': '234'})
HTTP response body: {"code":"unprocessable entity","message":"failure writing points to database: partial write: field type conflict: input field \"TANGARA_48C6\" on measurement \"AQI_LAST_HOUR\" is type integer, already exists as type float dropped=24"}



# Extraction - AQI Last 8 Hours

In [20]:
# Kedro Catalog
aqi_last_8h = catalog.load('aqi_last_8h')
# Data Frame Sensors
aqi_last_8h.head()

2022-09-17 17:08:03,246 - kedro.io.data_catalog - INFO - Loading data from `aqi_last_8h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 01:06:00,59,64,66,42,52,37,4,16,21,44,18,56
1,2022-09-17 09:06:00,63,71,77,58,65,46,10,38,62,61,49,66
2,2022-09-17 17:06:00,38,43,52,41,40,31,11,34,57,51,43,46


In [21]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(aqi_last_8h, 'AQI_LAST_8H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(aqi_last_8h, 'AQI_LAST_8H')

# Extraction - AQI Last 12 Hours

In [22]:
# Kedro Catalog
aqi_last_12h = catalog.load('aqi_last_12h')
# Data Frame Sensors
aqi_last_12h.head()

2022-09-17 17:08:07,609 - kedro.io.data_catalog - INFO - Loading data from `aqi_last_12h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 05:06:00,61,66,69,48,57,36,5,18,28,48,23,59
1,2022-09-17 17:06:00,49,56,61,51,52,40,12,40,63,57,50,54


In [23]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(aqi_last_12h, 'AQI_LAST_12H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(aqi_last_12h, 'AQI_LAST_12H')

# Extraction - AQI Last 24 Hours

In [24]:
# Kedro Catalog
aqi_last_24h = catalog.load('aqi_last_24h')
# Data Frame Sensors
aqi_last_24h.head()

2022-09-17 17:08:10,802 - kedro.io.data_catalog - INFO - Loading data from `aqi_last_24h` (CSVDataSet)...


,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-17 17:06:00,56,61,65,49,54,38,8,29,52,53,37,57


In [25]:
# Ingesting each Tangara Sensor DataFrame to InfluxDB
ingesting_influxdb(aqi_last_24h, 'AQI_LAST_24H')
# Ingesting each Tangara Sensor DataFrame to InfluxDB 1.8
ingesting_influxdb_1_8(aqi_last_24h, 'AQI_LAST_24H')